In [ ]:
!pip install pygame

In [ ]:
import cv2
import numpy as np
import pygame
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Initialize pygame
pygame.init()

# Load model pengenalan wajah dan label encoder
model = load_model("face_recognition_model.h5")
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("label_encoder.npy", allow_pickle=True)

# Buka kamera
camera = cv2.VideoCapture(0)
if not camera.isOpened():
    print("Error: Kamera tidak dapat diakses")
    sys.exit(1)

# Load Haar Cascade untuk deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("Tekan tombol 'q' untuk keluar")

try:
    screen = pygame.display.set_mode((640, 480))
    pygame.display.set_caption("Face Recognition")

    running = True
    while running:
        ret, frame = camera.read()
        if not ret:
            print("Gagal membaca frame dari kamera")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Deteksi wajah pada frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Ekstrak wajah
                face_roi = gray[y:y + h, x:x + w]
                face_roi = cv2.resize(face_roi, (64, 64))

                # Normalisasi dan prediksi menggunakan model
                normalized_face = face_roi / 255.0
                reshaped_face = np.reshape(normalized_face, (1, 64, 64, 1))
                result = model.predict(reshaped_face)
                label = label_encoder.classes_[np.argmax(result)]

                # Tambahkan label dan kotak pada wajah
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Convert frame to Pygame surface
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_surface = pygame.image.fromstring(frame_rgb.tobytes(), frame_rgb.shape[1::-1], "RGB")

        # Rotate and scale the frame for pygame
        frame_rotated = pygame.transform.rotate(frame_surface, 360)
        frame_scaled = pygame.transform.scale(frame_rotated, (640, 480))

        # Display the frame
        screen.blit(frame_scaled, (0, 0))
        pygame.display.flip()

        # Keluar dari loop dengan menekan tombol 'q'
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_q):
                running = False

finally:
    camera.release()
    pygame.quit()
    print("Kamera dimatikan")


Tekan tombol 'q' untuk keluar
1/1 [==============================] - 0s 27ms/step
Kamera dimatikan
